In [1]:
#Dependencies
from matplotlib import pyplot as plt
import pandas as pd
import sklearn
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.cluster import KMeans
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [2]:
#read csv into df
df = pd.read_csv("Levels_Fyi_Salary_Data 2.csv")
# columns to drop... salary_currency, salary, work_year
df.sample(25)

,timestamp,company,level,title,totalyearlycompensation,location,yearsofexperience,yearsatcompany,tag,basesalary,...,Doctorate_Degree,Highschool,Some_College,Race_Asian,Race_White,Race_Two_Or_More,Race_Black,Race_Hispanic,Race,Education
49616,4/26/2021 10:13:54,PNC,Technology Development Program,Software Engineer,85000,"Pittsburgh, PA",0.0,0.0,Rotational,65000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
62053,7/13/2018 14:38:27,Amazon,6,Software Engineering Manager,500000,"Seattle, WA",20.0,16.0,iOS,160000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
52363,5/25/2021 10:06:58,Match,Lead Software Engineer,Software Engineer,140000,"Dallas, TX",11.0,5.0,Web Development (Front-End),125000.0,...,0,0,0,0,1,0,0,0,White,Bachelor's Degree
23161,6/26/2020 23:44:30,Amazon,IC6,Product Manager,225000,"Seattle, WA",11.0,3.0,Full Stack,136000.0,...,0,0,0,0,0,0,0,0,NaN,Master's Degree
6664,5/21/2019 22:22:42,Salesforce,MTS,Software Engineer,178000,"San Francisco, CA",0.0,0.0,Distributed Systems (Back-End),142000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
56288,7/1/2021 10:32:11,Microsoft,67,Software Engineering Manager,480000,"Seattle, WA",25.0,20.0,Full Stack,220000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
12476,10/28/2019 20:42:51,Uber,4,Software Engineer,263000,"Palo Alto, CA",4.0,4.0,Distributed Systems (Back-End),150000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
61301,8/12/2021 21:49:29,Square,Level 4,Software Engineer,233000,"San Francisco, CA",2.0,0.0,API Development (Back-End),150000.0,...,0,0,0,0,1,0,0,0,White,Master's Degree
46128,3/26/2021 7:16:32,Intel,Grade 9,Product Designer,290000,"Santa Clara, CA",25.0,5.0,Communication,200000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
9348,8/1/2019 3:47:08,Expedia,L,Software Engineer,196000,"San Francisco, CA",3.0,3.0,Mobile (iOS + Android),162000.0,...,0,0,0,0,0,0,0,0,NaN,NaN


In [3]:
df.nunique()


timestamp                  62561
company                     1631
level                       2923
title                         15
totalyearlycompensation      893
location                    1050
yearsofexperience             65
yearsatcompany                81
tag                         3058
basesalary                   482
stockgrantvalue              612
bonus                        335
gender                         4
otherdetails               12841
cityid                      1045
dmaid                        149
rowNumber                  62642
Masters_Degree                 2
Bachelors_Degree               2
Doctorate_Degree               2
Highschool                     2
Some_College                   2
Race_Asian                     2
Race_White                     2
Race_Two_Or_More               2
Race_Black                     2
Race_Hispanic                  2
Race                           5
Education                      5
dtype: int64

In [4]:
titles_df = df[df["title"]=="Data Scientist"]
titles_df


,timestamp,company,level,title,totalyearlycompensation,location,yearsofexperience,yearsatcompany,tag,basesalary,...,Doctorate_Degree,Highschool,Some_College,Race_Asian,Race_White,Race_Two_Or_More,Race_Black,Race_Hispanic,Race,Education
419,6/5/2018 14:06:30,LinkedIn,Senior,Data Scientist,233000,"San Francisco, CA",4.0,0.0,Data Analysis,162000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
440,6/8/2018 9:49:25,Microsoft,64,Data Scientist,218000,"Seattle, WA",11.0,11.0,ML / AI,165000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
444,6/8/2018 17:55:09,ebay,26,Data Scientist,180000,"San Jose, CA",10.0,5.0,NaN,0.0,...,0,0,0,0,0,0,0,0,NaN,NaN
454,6/10/2018 19:39:35,Twitter,Staff,Data Scientist,500000,"San Francisco, CA",4.0,4.0,ML / AI,200000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
495,6/17/2018 11:39:38,Facebook,5,Data Scientist,370000,"Seattle, WA",8.0,3.0,NaN,190000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62240,3/26/2019 17:25:02,IBM,8,Data Scientist,155000,"Washington, DC",5.0,2.0,ML / AI,141000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
62283,6/17/2018 21:07:18,Facebook,E3,Data Scientist,150000,"Singapore, SG, Singapore",0.0,0.0,ML / AI,150000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
62285,7/16/2018 20:11:40,ADP,Sr UX Researcher 4,Data Scientist,185000,"Roseland, NJ",8.0,2.0,UX Research,150000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
62529,6/12/2018 20:54:06,Google,T6,Data Scientist,685000,"Kirkland, WA",22.0,2.0,ML / AI,221000.0,...,0,0,0,0,0,0,0,0,NaN,NaN


In [5]:
locations = titles_df["location"]
locations.value_counts()

San Francisco, CA    373
Seattle, WA          311
New York, NY         245
Redmond, WA          105
Menlo Park, CA        94
                    ... 
Columbia, MO           1
Campbell, CA           1
Italy, TX              1
Titusville, NJ         1
Roseland, NJ           1
Name: location, Length: 225, dtype: int64

In [6]:
clean_df= titles_df.drop(columns=['timestamp', 'level', 'yearsatcompany', 'otherdetails','Doctorate_Degree', 'Bachelors_Degree',
                              'Highschool', 'Some_College', 'Race_Asian', 'Race_White', 'Race_Two_Or_More', 'Race_Black',
                              'Race_Hispanic', 'Race', 'Education','cityid','dmaid','rowNumber','Masters_Degree','tag','bonus','stockgrantvalue','basesalary','title'])
clean_df.head()
#get_dummies/oneHotEncoder on company size and employment type

,company,totalyearlycompensation,location,yearsofexperience,gender
419,LinkedIn,233000,"San Francisco, CA",4.0,Male
440,Microsoft,218000,"Seattle, WA",11.0,Male
444,ebay,180000,"San Jose, CA",10.0,Female
454,Twitter,500000,"San Francisco, CA",4.0,Male
495,Facebook,370000,"Seattle, WA",8.0,Male


In [7]:
clean_df.nunique()

company                    467
totalyearlycompensation    420
location                   225
yearsofexperience           28
gender                       3
dtype: int64

In [11]:
print(clean_df["company"].value_counts().head(10))

Amazon          257
Microsoft       217
Facebook        184
Google           89
Apple            78
IBM              66
Uber             60
Capital One      47
Walmart Labs     41
LinkedIn         36
Name: company, dtype: int64


In [27]:
#Binning he companies- .cut with 5 bins...do we want to specifically design these?
bin_labels = ['Small', 'Medium', 'Large', 'Very Large']
bins4 = clean_df['company'].value_counts()
company_bins = pd.cut(bins4, bins=4, labels=bin_labels).to_list()#does this solve the issue of imbalanced data?
company_bins

['Very Large',
 'Very Large',
 'Large',
 'Medium',
 'Medium',
 'Medium',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small

In [33]:
#change company name to bin name
companies = []
for company in clean_df['company']:
    if company == "Amazon" or company == "Microsoft":
        company = "Very Large"
    elif company == "Facebook":
        company = "Large"
    elif company == "Google" or company == "Apple" or company == "IBM":
        company = "Medium"
    else:
        company = "Small"
    companies.append(company)

In [82]:
#reassign company column to bin name
clean_bins_df = clean_df.copy()
clean_bins_df['company'] = companies
clean_bins_df.head(10)
clean_bins_df['gender'].value_counts()

Male      1357
Female     361
Other       22
Name: gender, dtype: int64

In [72]:
#is their linearity between location numerical representation and salary?
clean_bins2 = clean_bins_df.drop(['location'], axis = 1)
locations = clean_bins_df['location']
clean_bins2.head()

,company,totalyearlycompensation,yearsofexperience,gender
419,Small,233000,4.0,Male
440,Very Large,218000,11.0,Male
444,Small,180000,10.0,Female
454,Small,500000,4.0,Male
495,Large,370000,8.0,Male


In [73]:
#Create label Encoder and encode categorical
from sklearn.preprocessing import LabelEncoder
lb_make = LabelEncoder()
clean_bins2['company']= lb_make.fit_transform(clean_bins2['company'])
clean_bins2['gender']= lb_make.fit_transform(clean_bins2['gender'])
clean_bins2.head()

,company,totalyearlycompensation,yearsofexperience,gender
419,2,233000,4.0,1
440,3,218000,11.0,1
444,2,180000,10.0,0
454,2,500000,4.0,1
495,0,370000,8.0,1


In [76]:
clean_bins2['gender'].value_counts() #what do we do with 4 genders?

1    1357
3     838
0     361
2      22
Name: gender, dtype: int64

In [68]:
#OneHot to new categories
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(sparse=False)
company_encoded = lb_make.fit_transform(clean_bins2['company'])
company_encoded = company_encoded.reshape(len(company_encoded), 1)
onehot_company_encoded = enc.fit_transform(company_encoded)
print(onehot_company_encoded)

[[0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 ...
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]


In [69]:
# clean_bins_enc = .copy()
# clean_bins_enc['company'] = onehot_company_encoded
# clean_bins_enc.head()

SyntaxError: invalid syntax (749818807.py, line 1)

In [83]:
#skip procedural with get_dummies()?
# clean_dummies = clean_dummies.drop(['gender_2'], axis=1)
clean_dummies = pd.get_dummies(data=clean_bins2, columns=['company', 'gender'])#drop the gender columns?
clean_dummies.head(15)

,totalyearlycompensation,yearsofexperience,company_0,company_1,company_2,company_3,gender_0,gender_1,gender_2,gender_3
419,233000,4.0,0,0,1,0,0,1,0,0
440,218000,11.0,0,0,0,1,0,1,0,0
444,180000,10.0,0,0,1,0,1,0,0,0
454,500000,4.0,0,0,1,0,0,1,0,0
495,370000,8.0,1,0,0,0,0,1,0,0
499,200000,3.0,0,0,0,1,0,1,0,0
509,340000,11.0,0,0,0,1,0,1,0,0
510,690000,10.0,0,1,0,0,0,1,0,0
513,600000,3.0,0,0,1,0,0,1,0,0
523,168000,8.0,0,0,1,0,0,1,0,0


In [78]:
X = clean_dummies.drop(columns=["totalyearlycompensation"])
y = clean_dummies["totalyearlycompensation"]
X.shape
# Scale the dataset using MinMaxScaler()
X_scaled = MinMaxScaler().fit_transform(X)
X_scaled

array([[0.11764706, 0.33333333, 0.        , 0.        , 1.        ,
        0.        ],
       [0.32352941, 0.33333333, 0.        , 0.        , 0.        ,
        1.        ],
       [0.29411765, 0.        , 0.        , 0.        , 1.        ,
        0.        ],
       ...,
       [0.23529412, 0.33333333, 0.        , 0.        , 1.        ,
        0.        ],
       [0.64705882, 0.33333333, 0.        , 1.        , 0.        ,
        0.        ],
       [0.17647059, 1.        , 0.        , 0.        , 1.        ,
        0.        ]])

In [79]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_scaled, 
                                                    y, 
                                                    random_state=42)

In [ ]:
# from sklearn.datasets import make_regression

# n_features = 3
# X, y = make_regression(n_features=n_features, 
#                        n_informative=n_features, random_state=1, 
#                        noise=0.5, bias=100.0)
# print(X.shape)

In [ ]:
# from mpl_toolkits.mplot3d import Axes3D
# fig = plt.figure(1, figsize=(5, 5))
# axes = Axes3D(fig, elev=20, azim=45)
# axes.scatter(X[:,0], X[:,1], X[:,2], c=y, cmap=plt.cm.get_cmap("Spectral"))
# plt.show()


In [80]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

# Fitting our model with all our features in X
model.fit(X_train, y_train)
training_score = model.score(X_train, y_train)
testing_score = model.score(X_test, y_test)


print(f"Training Score: {training_score}")
print(f"Testing Score: {testing_score}")

Training Score: 0.15494256360154512
Testing Score: 0.16597498001429778


In [ ]:
# #Residuals
# predictions = model.predict(X)
# # Plot Residuals
# plt.scatter(predictions, predictions - y)
# plt.hlines(y=0, xmin=predictions.min(), xmax=predictions.max())
# plt.show()


In [90]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=42)

In [91]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
# Fitting Random Forest Regression to the dataset
# import the regressor
from sklearn.ensemble import RandomForestRegressor
 
 # create regressor object
regressor = RandomForestRegressor(n_estimators = 100, random_state = 42)
 
# fit the regressor with x and y data
regressor.fit(X, y) 

RandomForestRegressor(random_state=42)

In [92]:
Y_pred = regressor.predict(X_test)  # test the output by changing values


In [94]:
training_score = regressor.score(X_train, y_train)
testing_score = regressor.score(X_test, y_test)


print(f"Training Score: {training_score}")
print(f"Testing Score: {testing_score}")

Training Score: 0.25946769966654326
Testing Score: 0.28260620926837376
